In [1]:
import torch
from run_netf_helpers import *
from scipy import io
import numpy as np

seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/public/home/shensy/.conda/envs/nerfnlos/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/public/home/shensy/.conda/envs/nerfnlos/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
model = torch.load("./logs/pikachu/run_netf_pikachu_decoloar/model/model_50000.pt")
# model.eval()

model.to(device)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
magic_number = 1 / 2
camera_grid_points = 64
camera_grid_positions_z = np.linspace(-magic_number, magic_number, camera_grid_points)
camera_grid_positions_z = np.outer(camera_grid_positions_z, np.ones_like(camera_grid_positions_z))
camera_grid_positions_y = np.zeros([camera_grid_points*camera_grid_points,1])
camera_grid_positions_x = np.linspace(-magic_number, magic_number, camera_grid_points)
camera_grid_positions_x = np.outer(np.ones_like(camera_grid_positions_x), camera_grid_positions_x)
camera_grid_positions_z = camera_grid_positions_z.flatten().reshape([-1,1])
camera_grid_positions_x = camera_grid_positions_x.flatten().reshape([-1,1])
camera_grid_positions = np.concatenate((camera_grid_positions_x, camera_grid_positions_y, camera_grid_positions_z), axis=1)
camera_grid_positions = camera_grid_positions.swapaxes(0,1)


v_light = 3 
start = 150
end = 1048
data_start = start
data_end =  end
wall_size = 0.55
deltaT = 0.0012 / 2

pmin = torch.Tensor([-wall_size/2 - data_end * deltaT, - data_end * deltaT, -wall_size/2 - data_end * deltaT, 0, -np.pi]).float()
pmax = torch.Tensor([wall_size/2 + data_end * deltaT, - data_start * deltaT, wall_size/2 + data_end * deltaT, np.pi, 0]).float()

# res = np.zeros(camera_grid_positions.shape[1]*(end-start))
res = np.zeros([camera_grid_positions.shape[1]*(end- start), 2])

reso = 64
res = np.zeros([reso ^ 3, 2])

In [ ]:
input_x, input_y, input_z = torch.meshgrid(
    torch.linspace(-magic_number, magic_number, reso),
    torch.linspace(data_start * deltaT, data_end * deltaT, reso) * -1,
    torch.linspace(-magic_number, magic_number, reso)
)

input_x = input_x.reshape([-1, 1])
input_y = input_y.reshape([-1, 1])
input_z = input_z.reshape([-1, 1])

test_input = torch.cat((input_x, input_y, input_z, torch.ones_like(input_x) * np.pi/2, torch.ones_like(input_x) * np.pi/2 * -1), axis = 1)
test_input = (test_input - pmin) / (pmax - pmin)
test_input_pe = encoding_sph_tensor(test_input.to('cuda:0'), 4, 4, False)


In [ ]:

output = model(test_input_pe)

# io.savemat('./test.mat', {'mua_grid': output.cpu().data.numpy().squeeze()})

In [ ]:
rho = output[:,0]
alpha = output[:,1]

In [ ]:
rho = rho.reshape([reso, reso, reso])
alpha = alpha.reshape([reso, reso, reso])

In [ ]:
import matplotlib.pyplot as plt

rho_img = rho.max(axis = 1).values
plt.imshow(rho_img.cpu().data.numpy().squeeze())


In [ ]:
alpha_img = alpha.max(axis = 1).values
plt.imshow(alpha_img.cpu().data.numpy().squeeze())

In [ ]:
raw_img = alpha_img.mul(rho_img)
plt.imshow(raw_img.cpu().data.numpy().squeeze())

In [ ]:
from scipy import io 

io.savemat('./vol.mat', {'rho': rho.cpu().data.numpy().squeeze(), 'alpha': alpha.cpu().data.numpy().squeeze()})